In [1]:

import numpy as np
import cv2 as cv
import glob


In [2]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('.\images\*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(1000)
        
cv.destroyAllWindows()

In [3]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(ret)
print(mtx)
print(dist)
print(rvecs)
print(tvecs)

0.155369069100788
[[534.07088367   0.         341.53407538]
 [  0.         534.11914599 232.94565269]
 [  0.           0.           1.        ]]
[[-2.92971637e-01  1.07706963e-01  1.31038377e-03 -3.11018871e-05
   4.34798099e-02]]
[array([[-0.43239599],
       [ 0.25603401],
       [-3.08832021]]), array([[ 0.41531697],
       [ 0.65664497],
       [-1.3373494 ]]), array([[-0.2645143 ],
       [-0.39360849],
       [-2.74787379]]), array([[-0.37843358],
       [-0.18064237],
       [-3.11615996]]), array([[-0.45993978],
       [-0.3142018 ],
       [-1.76122223]]), array([[-0.29979221],
       [ 0.39216377],
       [-1.4348239 ]]), array([[-0.32034625],
       [ 0.1597993 ],
       [-1.24149595]]), array([[-0.45883216],
       [-0.08848877],
       [-1.33510786]]), array([[-0.35367631],
       [-0.24363035],
       [-1.56874295]]), array([[ 0.30697385],
       [ 0.5038552 ],
       [-1.82824733]]), array([[-0.17288944],
       [-0.46764681],
       [ 1.34745198]])]
[array([[ 3.79739147

In [5]:
img = cv.imread('.\images\left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [6]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True

In [7]:
import sys
print(sys.path)

['C:\\Users\\Pedro\\Desktop\\brasis\\Calibración de cámara', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara\\python38.zip', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara\\DLLs', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara\\lib', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara', '', 'C:\\Users\\Pedro\\AppData\\Roaming\\Python\\Python38\\site-packages', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara\\lib\\site-packages', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara\\lib\\site-packages\\win32', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Pedro\\.conda\\envs\\Calibracion de Camara\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Pedro\\.ipython']


In [7]:
cv.imshow('calibresult.png', dst)
cv.imshow('Original image', img)
cv.waitKey(500)

-1

In [8]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.023686000375385676
